In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from keras.datasets.cifar10 import load_data
(x_train, y_train), (x_test, y_test) = load_data()

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
Using TensorFlow backend.


In [2]:
#validation set 나누기
x_val,y_val = x_train[-10000:], y_train[-10000:]
x_train, y_train = x_train[:40000], y_train[:40000]

In [3]:
def next_batch(num, data, labels):
    idx = np.arange(0,len(data))
    np.random.shuffle(idx)
    idx=idx[:num]
    data_shuffle=[data[i] for i in idx]
    labels_shuffle=[labels[i] for i in idx]
    
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [5]:
learning_rate = 0.001
training_epoch=10
batch_size=128
n_classes=10

In [6]:
class Inception():
    def __init__(self,X,kernel_1_out,kernel_3_in,kernel_3_out,kernel_5_in,kernel_5_out,pool_out):
        #1x1
        self.L1 = tf.layers.conv2d(X,kernel_1_out,[1,1],padding='SAME', activation=tf.nn.relu)
        self.L1 = tf.layers.batch_normalization(self.L1)
        
        #1x1 -> 3x3
        self.L2 = tf.layers.conv2d(X,kernel_3_in,[1,1],padding='SAME', activation=tf.nn.relu)
        self.L2 = tf.layers.batch_normalization(self.L2)
        self.L2 = tf.layers.conv2d(self.L2,kernel_3_out,[3,3],padding='SAME', activation=tf.nn.relu)
        self.L2 = tf.layers.batch_normalization(self.L2)
        
        #1x1 -> 5x5
        self.L3 = tf.layers.conv2d(X,kernel_5_in,[1,1],padding='SAME', activation=tf.nn.relu)
        self.L3 = tf.layers.batch_normalization(self.L3)
        self.L3 = tf.layers.conv2d(self.L3,kernel_5_out,[3,3],padding='SAME', activation=tf.nn.relu)
        self.L3 = tf.layers.batch_normalization(self.L3)
        self.L3 = tf.layers.conv2d(self.L3,kernel_5_out,[3,3],padding='SAME', activation=tf.nn.relu)
        self.L3 = tf.layers.batch_normalization(self.L3)
        
        #3x3max_pool -> 3x3
        self.L4 = tf.layers.max_pooling2d(X,[3,3],[1,1],padding='SAME')
        self.L4 = tf.layers.conv2d(X,pool_out,[3,3],padding='SAME', activation=tf.nn.relu)
        self.L4 = tf.layers.batch_normalization(self.L4)
        
    def forward(self):
        return tf.concat([self.L1,self.L2,self.L3,self.L4],axis=3)

class GoogLenet():
    def __init__(self,sess):
        self.sess=sess
        self.X = tf.placeholder(tf.float32,[None,32,32,3])
        self.Y = tf.placeholder(tf.int64,[None,1])
        self.Y_one_hot = tf.one_hot(self.Y,10)
        self.Y_one_hot = tf.reshape(self.Y_one_hot,[-1,10])
        self.build_net()
        
    def build_net(self):
        self.pre=tf.layers.conv2d(self.X,192,[3,3],padding='SAME',activation=tf.nn.relu)
        self.pre=tf.layers.batch_normalization(self.pre)
        
        #input, 1x1out, 3x3in, 3x3out, 5x5in, 5x5out, pool_out
        self.g1=Inception(self.pre,64,96,128,16,32,32)#64+128+32+32=256 concat후 채널수
        self.g2=Inception(self.g1.forward(),128,128,192,32,96,64)
        self.max_pool1=tf.layers.max_pooling2d(self.g2.forward(),[2,2],[2,2],padding='SAME')
    
        self.g3=Inception(self.max_pool1,32,32,32,32,32,32)
        self.g4=Inception(self.g3.forward(),64,32,32,32,32,32)
        self.g5=Inception(self.g4.forward(),64,32,32,32,32,32)
        self.g6=Inception(self.g5.forward(),64,32,32,32,32,32)
        self.g7=Inception(self.g6.forward(),64,32,32,32,32,32)
        self.max_pool2=tf.layers.max_pooling2d(self.g7.forward(),[2,2],[2,2],padding='SAME')
    
        self.g8=Inception(self.max_pool2,64,32,32,32,32,32)
        self.g9=Inception(self.g8.forward(),64,32,32,32,32,32)
        
        self.avg_pool=tf.layers.average_pooling2d(self.g9.forward(),[8,8],[1,1],padding='SAME')
        
        last_L=tf.layers.flatten(self.avg_pool)
        FC = tf.layers.dense(last_L,512,activation=tf.nn.relu)
        self.logits = tf.layers.dense(FC,10,activation=None)

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits,labels=self.Y_one_hot))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)

        is_correct=tf.equal(tf.argmax(self.logits,1),tf.argmax(self.Y_one_hot,1))
        self.accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
        
    def train(self,x_data,y_data):
        return self.sess.run([self.cost,self.optimizer],feed_dict={self.X:x_data,self.Y:y_data})
    
    def get_accuracy(self,x_test,y_test):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test,self.Y:y_test})
    
    def prediction(self,x_test):
        return self.sess.run(tf.argmax(self.logits,1),feed_dict={self.X:x_test})


In [7]:
#Saver
global_step = tf.Variable(0, trainable=True, name='global_step')#초기값0 trainable-카운트
sess=tf.Session()

#세션 생성후 모델생성
model= GoogLenet(sess)

saver = tf.train.Saver(tf.global_variables())#앞서 정의한 변수를 가져오는 함수
ckpt = tf.train.get_checkpoint_state('./Saver/GoogLenet001')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):# if you want to skip, add and 0 
    saver.restore(sess, ckpt.model_checkpoint_path)
    print('Load Success')
else:
    sess.run(tf.global_variables_initializer())
    print('Load Fail')

Load Fail


In [8]:
total_batch = int(x_train.shape[0]/batch_size)

print('Learning Start')

max_val_acc = 0
for epoch in range(10):
    c_avg = 0
    for i in range(total_batch):
        batch_xs, batch_ys = next_batch(batch_size,x_train,y_train)
        c,_=model.train(batch_xs,batch_ys)
        c_avg+=c/total_batch
        
    
    val_acc=0
    train_acc=0
    for i in range(training_epoch):
        offset = i*1000
        x_s,y_s=x_test[offset:offset+1000],y_test[offset:offset+1000]
        val_acc_tmp=model.get_accuracy(x_s,y_s)
        val_acc+=val_acc_tmp/10
        
    for i in range(10):
        x_s,y_s=next_batch(batch_size,x_train,y_train)
        train_acc_tmp=model.get_accuracy(x_s,y_s)
        train_acc+=train_acc_tmp/10
        
    if max_val_acc<val_acc:
        max_val_acc=val_acc
        saver.save(sess, './Saver/GoogLenet01/GoogLenet01.ckpt',global_step = global_step)
    print('Epoch: {:03d}'.format(epoch+1),'Cost:%.7f'%c_avg,'Val_Acc:{:.5f}'.format(val_acc),'Train_Acc:{:5f}'.format(train_acc))
    
    
print('Max_val_acc:{:.5f}'.format(max_val_acc))    
print('Learning Finished')    

Learning Start
Epoch: 001 Cost:1.9277818 Val_Acc:0.42030 Train_Acc:0.421875
Epoch: 002 Cost:1.3649510 Val_Acc:0.56610 Train_Acc:0.580469
Epoch: 003 Cost:1.0804281 Val_Acc:0.63230 Train_Acc:0.665625
Epoch: 004 Cost:0.8732932 Val_Acc:0.69960 Train_Acc:0.729687
Epoch: 005 Cost:0.7349302 Val_Acc:0.71590 Train_Acc:0.757031
Epoch: 006 Cost:0.6361264 Val_Acc:0.71430 Train_Acc:0.757812
Epoch: 007 Cost:0.5495704 Val_Acc:0.73340 Train_Acc:0.820312
Epoch: 008 Cost:0.4912168 Val_Acc:0.75150 Train_Acc:0.828125
Epoch: 009 Cost:0.4427347 Val_Acc:0.76030 Train_Acc:0.867188
Epoch: 010 Cost:0.3956081 Val_Acc:0.77380 Train_Acc:0.875000
Max_val_acc:0.77380
Learning Finished


In [9]:
#Val_Acc 기준으로 가장 좋은모델 불러오기
ckpt = tf.train.get_checkpoint_state('./Saver/GoogLenet01')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):# if you want to skip, add and 0 
    saver.restore(sess, ckpt.model_checkpoint_path)
    print('Load Success')
else:
    sess.run(tf.global_variables_initializer())
    print('Load Fail')

INFO:tensorflow:Restoring parameters from ./Saver/GoogLenet01/GoogLenet01.ckpt-0
Load Success


In [10]:
#전체 테스트셋에서의 정확도 OOM때문에 분할계산 후 평균
acc_avg=0
test_size = x_test.shape[0]
total_batch = int(test_size/batch_size)

for i in range(total_batch):
    offset = i*batch_size
    x_s,y_s=x_test[offset:offset+batch_size],y_test[offset:offset+batch_size]
    c=model.get_accuracy(x_s,y_s)
    acc_avg+=c
print(acc_avg/total_batch)

0.7740384615384616


In [21]:
#랜덤한 수로 하나 뽑음
import random
a=random.randint(0,x_test.shape[0]-1)
print("실제값:",y_test[a:a+1])
print("예측값:",model.prediction(x_test[a:a+1]))

실제값: [[9]]
예측값: [9]
